In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Input
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

2024-04-26 08:09:59.867369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 08:09:59.867549: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 08:10:00.040060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
tweets = train_data['text'].values
labels = train_data['target'].values

In [3]:
x_train, x_val, y_train, y_val = train_test_split(tweets, labels, test_size=0.3, random_state=42, shuffle=True, stratify=None)
x_val, x_test1, y_val, y_test1 = train_test_split(x_val, y_val, test_size=0.1, random_state=42, shuffle=True, stratify=None)

In [4]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweets)

In [5]:
x_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(x_train, maxlen=350)
x_val = tokenizer.texts_to_sequences(x_val)
x_val = pad_sequences(x_val, maxlen=350)
x_test1 = tokenizer.texts_to_sequences(x_test1)
x_test1 = pad_sequences(x_test1, maxlen=350)

In [6]:
model = Sequential()
model.add(Input((x_train.shape[1],)))
model.add(Embedding(5000, 100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(150, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l1(0.001)))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 350, 100)       │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 350, 100)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           151 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650,751 (2.48 MB)

 Trainable params: 650,751 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.fit(x_train, y_train, epochs=5, batch_size=44, validation_data=(x_val, y_val))

Epoch 1/5
122/122 ━━━━━━━━━━━━━━━━━━━━ 65s 506ms/step - accuracy: 0.5893 - loss: 0.6782 - val_accuracy: 0.7382 - val_loss: 0.5473
Epoch 2/5
122/122 ━━━━━━━━━━━━━━━━━━━━ 61s 501ms/step - accuracy: 0.7810 - loss: 0.4890 - val_accuracy: 0.8058 - val_loss: 0.4617
Epoch 3/5
122/122 ━━━━━━━━━━━━━━━━━━━━ 80s 489ms/step - accuracy: 0.8531 - loss: 0.3767 - val_accuracy: 0.8151 - val_loss: 0.4643
Epoch 4/5
122/122 ━━━━━━━━━━━━━━━━━━━━ 84s 506ms/step - accuracy: 0.8772 - loss: 0.3313 - val_accuracy: 0.7971 - val_loss: 0.4861
Epoch 5/5
122/122 ━━━━━━━━━━━━━━━━━━━━ 81s 496ms/step - accuracy: 0.9016 - loss: 0.2795 - val_accuracy: 0.8068 - val_loss: 0.5235


In [9]:
y_pred = model.predict(x_test1)
y_pred = np.rint(y_pred)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step


In [10]:
print('Accuracy: {:.3f}'.format(accuracy_score(y_test1, y_pred)))
print("F1 Score:", f1_score(y_test1, y_pred))

Accuracy: 0.825
F1 Score: 0.7802197802197802


In [11]:
test_data = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [12]:
test_data.isna().any()

id          False
keyword      True
location     True
text        False
dtype: bool

In [13]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [14]:
tweets = test_data['text']

In [15]:
len(tweets)

3263

In [16]:
x_test = tokenizer.texts_to_sequences(tweets)
x_test = pad_sequences(x_test, maxlen=350)

In [17]:
y_pred = model.predict(x_test)
y_pred = np.rint(y_pred)

102/102 ━━━━━━━━━━━━━━━━━━━━ 13s 132ms/step


In [18]:
y_pred

array([[0.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [19]:
df_result = pd.DataFrame(test_data['id'], columns=['id'])
df_result['target'] = y_pred.astype(int)
df_result

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [20]:
df_result.to_csv("/kaggle/working/submission.csv", index=False)